In [8]:
# SEO环境： schedule
import os
import warnings
warnings.filterwarnings('ignore')
import requests
import pandas as pd
import numpy as np
import jsonpath
import json
import time
import datetime
import xlwings as xw
import telebot
import hmac, base64, struct, hashlib
from openpyxl import Workbook, load_workbook
from openpyxl import formatting, styles
from openpyxl.styles import Color, PatternFill, Font, Border
from PIL import ImageGrab
import pyperclip

pd.set_option('display.max_colwidth', None) #显示单元格完整信息
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

day = -1
pages_user = 150
pages_fircharge = 60
pages_fircharge_two = 100

# start_date = (datetime.datetime.now()+datetime.timedelta(days=day)).strftime('%Y%m%d')
# end_date = (datetime.datetime.now()+datetime.timedelta(days=day)).strftime('%Y%m%d')
# last_date = (datetime.datetime.now()+datetime.timedelta(days=day-1)).strftime('%Y%m%d')

url = 'http://fundmng.bsportsadmin.com/api/manage/fund/withdraw/record/list/history'

# 第一次获取token
submit_url = 'http://fundmng.bsportsadmin.com/api/manage/user/admin/login/submit'
header0 = {
    'Accept':'application/json, text/plain, */*',
    # 'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.9',
    'Connection':'keep-alive',
    'Content-Length':'48',
    'Content-Type':'application/x-www-form-urlencoded',
    'Cookie':'admin-uid=690; admin-token=db76bebda5274c80adaadd40bd794f24',
    'Device_id':'1.0',
    'Gl_version':'2.0',
    'Host':'fundmng.bsportsadmin.com',
    'Language':'zh_CN',
    'Origin':'http://fundmng.bsportsadmin.com',
    'Os_type':'0',
    'Referer':'http://fundmng.bsportsadmin.com/login',
    'Sign':'2bc4c378817f47731f0adf450a627d19',
    'Some':'header',
    'Systemid':"",
    'Timestamp':'1692415901000',
    'Token':'-1',
    'Uid':'-1',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
    'Version':'1.0'
}
def get_google_code(secret):
    key = base64.b32decode(secret, True)
    msg = struct.pack(">Q", int(time.time()) // 30)
    google_code = hmac.new(key, msg, hashlib.sha1).digest()
    # 很多网上的代码不可用，就在于这儿，没有chr字符串
    o = ord(chr(google_code[19])) & 15
    # google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    return '%06d' % google_code


# 采集会员列表和会员首存记录
url_fircharge = 'http://fundmng.bsportsadmin.com/api/manage/data/detail/firstRecharge'
url_user = 'http://fundmng.bsportsadmin.com/api/manage/user/maintain/user/list'
session = requests.session()

# 采集首存报表
dic_fir = dict({'会员名':[], '所属代理':[],'注册时间':[], '交易时间':[], '交易类型':[], '币种':[], '金额':[]})
# 昨天开始时间戳
yesterday = datetime.date.today() + datetime.timedelta(days=day)
yesterday_start_time = int(time.mktime(time.strptime(str(yesterday), '%Y-%m-%d')))
# 昨天结束时间戳
yesterday_end_time = int(time.mktime(time.strptime(str(datetime.date.today()), '%Y-%m-%d'))) - 1

#---------------------token----------------------
# 采取token
google_code = get_google_code('64ehnxj6yily5bhv23kgb62ozuh6yuu2')
data0 = {
    'username': 'Marquis',
    'password': 'qwer123456',
    'code': google_code
}
session0 = requests.Session()
response0  =session0.post(url=submit_url,data=data0,headers=header0)
response0.encoding = 'utf-8'
obj0 = json.loads(response0.text)
token = obj0['data']['token']

header = {
    'Accept':'application/json, text/plain, */*',
    # 'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.9',
    'Connection':'keep-alive',
    'Content-Length':'75',
    'Content-Type':'application/x-www-form-urlencoded',
    'Cookie':'admin-token=67c8b1bd1b434f898ed8570a860355b8; admin-uid=690',
    'Device_id':'1.0',
    'Gl_version':'2.0',
    'Host':'fundmng.bsportsadmin.com',
    'Language':'zh_CN',
    'Menuid':'100112',
    'Opeartionmenu':'%u62A5%u8868%u67E5%u8BE2-%u4F1A%u5458%u9996%u5B58%u62A5%u8868',
    'Origin':'http://fundmng.bsportsadmin.com',
    'Os_type':'0',
    'Referer':'http://fundmng.bsportsadmin.com/system/report-query/report-first-recharge',
    'Sign':'ca83944852acc68fe114cbc65f1e1d22',
    'Some':'header',
    'Systemid':'54',
    'Timestamp':'1692092554000',
    'Token':token,
    'Uid':'690',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Version':'1.0'
}
for page in range(1,pages_fircharge+1):
    data = {
        'page': page,
        'size': 20,
        'tradeType': 0,
        'isFake': 0,
        'stime': yesterday_start_time*1000,
        'etime': yesterday_end_time*1000+999
    }
    response = session.post(url_fircharge,headers=header,data=data)
    response.encoding = 'utf-8'
    obj = json.loads(response.text)

    for i in obj['data']['list']:
        dic_fir['会员名'].append(i['userName'])
        dic_fir['所属代理'].append(i['parentName'])
        dic_fir['注册时间'].append(time.strftime('%Y/%m/%d %H:%M:%S',time.localtime(i['createTime']//1000)))
        dic_fir['交易时间'].append(time.strftime('%Y/%m/%d %H:%M:%S',time.localtime(i['regTime']//1000)))
        dic_fir['交易类型'].append(i['tradeType'])
        dic_fir['币种'].append(i['coinCode'])
        dic_fir['金额'].append(i['amount'])
firChargeUser = pd.DataFrame(dic_fir)
print('会员首存行列：',firChargeUser.shape)

# 2.采集昨天和今天的首存报表
dic_fir_two = dict({'会员名':[], '所属代理':[],'注册时间':[], '交易时间':[], '交易类型':[], '币种':[], '金额':[]})
for page in range(1,pages_fircharge_two+1):
    data = {
        'page': page,
        'size': 20,
        'tradeType': 0,
        'isFake': 0,
        'stime': yesterday_start_time*1000,
        'etime': int(time.time())*1000
    }
    response = session.post(url_fircharge,headers=header,data=data)
    response.encoding = 'utf-8'
    obj = json.loads(response.text)

    for i in obj['data']['list']:
        dic_fir_two['会员名'].append(i['userName'])
        dic_fir_two['所属代理'].append(i['parentName'])
        dic_fir_two['注册时间'].append(time.strftime('%Y/%m/%d',time.localtime(i['createTime']//1000)))
        dic_fir_two['交易时间'].append(time.strftime('%Y/%m/%d',time.localtime(i['regTime']//1000)))
        dic_fir_two['交易类型'].append(i['tradeType'])
        dic_fir_two['币种'].append(i['coinCode'])
        dic_fir_two['金额'].append(i['amount'])
charge_two  = pd.DataFrame(dic_fir_two)
print('昨天与今天的会员首存行列：',charge_two.shape)

# 采集会员列表
dic_user = dict({'会员账号':[], '姓名':[],'代理':[],'玩家层级':[], '注册时间':[], '首存时间':[],'备注':[]})
for page in range(1,pages_user+1):
    data2 = {
        'page':page,
        'size':20,
        'userVip':'0,1,2,3,4,5,6,7,8,9,10,11',
        'status':'0,1,2,4',
        'sortType':'3',
        'sortStr':'descend',
        'searchType':'1',
        'channelId':'34',
        'registeredStartDate':yesterday_start_time*1000,
        'registeredEndDate':yesterday_end_time*1000+999,
    }
    response2 = session.post(url_user,headers=header,data=data2)
    response2.encoding = 'utf-8'
    obj2 = json.loads(response2.text)
    for i in obj2['data']['list']:
        dic_user['会员账号'].append(i['username'])
        if i['reallyName'] !='':
            dic_user['姓名'].append(i['reallyName'])
        else:
            dic_user['姓名'].append('-')

        dic_user['代理'].append(i['parentName'])
        dic_user['玩家层级'].append(i['levelName'])
        dic_user['注册时间'].append(time.strftime('%Y/%m/%d',time.localtime(i['registerDate']//1000)))
        dic_user['首存时间'].append(time.strftime('%Y/%m/%d',time.localtime(i['firstRechargeDate']//1000)))
        dic_user['备注'].append(i['remark'])
user = pd.DataFrame(dic_user)
print('用户列表行列:',user.shape)
# 删除测试账号
user = user[~user['会员账号'].str.contains('test')&~user['会员账号'].str.contains('ceshi')&~user['姓名'].str.contains('测试') \
            &~user['姓名'].str.contains('ceshi')&~user['代理'].str.contains('测试')&~user['代理'].str.contains('ceshi') \
            &~user['备注'].str.contains('测试')&~user['备注'].str.contains('试玩')&~user['备注'].str.contains('晒单') \
            &~user['玩家层级'].str.contains('测试')]
print('去重后：',user.shape)

shuju = pd.DataFrame({'人员':['paddy', 'tony', 'max', 'martin', 'zed', 'hugo', 'aber', 'dk', 'ben']})
shuju['人员2']=shuju['人员']
shuju.set_index('人员2',inplace=True)
shuju.sort_index(inplace=True)
# 注册数
daili = pd.read_excel(r'C:\Users\User\Desktop\SEO\SEO提单数据\1010\代理线.xlsx')
merge_user = pd.merge(user,daili,how = 'left',left_on='代理',right_on='代理线')
result = merge_user.groupby('seo变化数据团队').agg({'seo变化数据团队':len})
result.rename(columns={'seo变化数据团队':'注册数'},inplace=True)

# 绑卡
merge_user_dropna = merge_user.loc[~merge_user['seo变化数据团队'].isna()&(merge_user['姓名']!='-'),]
bangka = merge_user_dropna.groupby('seo变化数据团队').agg({'会员账号':len})
bangka.rename(columns={'会员账号':'绑卡数'},inplace=True)

# join1
result2 = result.join(bangka).fillna(0)
result2['绑卡率'] = result2['绑卡数']/result2['注册数']

trade = pd.read_csv(r'C:\Users\User\Desktop\SEO\SEO提单数据\1010\交易明细报表1010.csv',encoding='gbk')
huiyuan = pd.read_csv(r'C:\Users\User\Desktop\SEO\SEO提单数据\1010\流失会员详情.csv',encoding='gbk')

# 未拉过单
huiyuan2 = huiyuan.loc[huiyuan['首存时间']=='--',]
merge_huiyuan = huiyuan2.merge(daili,left_on = '代理',right_on='代理线')
merge_huiyuan2 = merge_huiyuan.loc[~merge_huiyuan['seo变化数据团队'].isna(),]
#名单
mingdan= set(list(merge_huiyuan2['会员账号'].values)+list(charge_two['会员名'].values))
# 删除测试账号
trade2 = trade[~trade['账户名'].str.contains('test')&~trade['账户名'].str.contains('ceshi')&~trade['所属代理'].str.contains('测试')&~trade['所属代理'].str.contains('ceshi')]
trade_daili= trade2.merge(daili,left_on='所属代理',right_on='代理线')

tidan = trade_daili.loc[trade_daili['账户名'].isin(mingdan),]
# 去除重复用户
tidan.drop_duplicates('账户名',inplace=True)
tidan_grp = tidan.groupby('seo变化数据团队').agg({'账户名':len}).rename(columns={'账户名':'提单人数'})
shuju = shuju.join(result2).join(tidan_grp)
shuju.fillna(0,inplace=True)
shuju['提单率']=shuju['提单人数']/shuju['绑卡数']

# 计算首存人数
charge_merge = firChargeUser.merge(daili,left_on = '所属代理',right_on='代理线')
charge_grp = charge_merge.groupby('seo变化数据团队').agg({'seo变化数据团队':len})
charge_grp.rename(columns={'seo变化数据团队':'首存数'},inplace=True)
shuju = shuju.join(charge_grp)
shuju.fillna(0,inplace=True)
shuju['提单充值率']=shuju['首存数']/shuju['提单人数']

# 当日注册当日首存
merge_2_user = merge_user.loc[merge_user['注册时间']==merge_user['首存时间'],]
merge_2_grp = merge_2_user.groupby('seo变化数据团队').agg({'seo变化数据团队':len})
merge_2_grp.rename(columns={'seo变化数据团队':'当日注册当日首存'},inplace=True)
shuju = shuju.join(merge_2_grp)
shuju.fillna(0,inplace=True)
shuju['当日注册当日首存率']  = shuju['当日注册当日首存']/shuju['注册数']

# 当天注册当天绑卡当天提单
merge_3_user = merge_user.loc[(merge_user['注册时间']==merge_user['首存时间'])&(~merge_user['seo变化数据团队'].isna())&(merge_user['姓名']!='-'),]
merge_3_grp = merge_3_user.groupby('seo变化数据团队').agg({'会员账号':len})
merge_3_grp.rename(columns={'会员账号':'当天注册当天绑卡当天提单'},inplace=True)
shuju = shuju.join(merge_3_grp)
shuju

会员首存行列： (410, 7)
昨天与今天的会员首存行列： (581, 7)
用户列表行列: (1118, 7)
去重后： (1022, 7)


,人员,注册数,绑卡数,绑卡率,提单人数,提单率,首存数,提单充值率,当日注册当日首存,当日注册当日首存率,当天注册当天绑卡当天提单
人员2,,,,,,,,,,,
aber,aber,37.0,14.0,0.378378,25,1.785714,13.0,0.520000,7.0,0.189189,6.0
ben,ben,5.0,2.0,0.400000,4,2.000000,3.0,0.750000,2.0,0.400000,2.0
dk,dk,13.0,2.0,0.153846,6,3.000000,3.0,0.500000,1.0,0.076923,1.0
hugo,hugo,55.0,15.0,0.272727,23,1.533333,17.0,0.739130,10.0,0.181818,9.0
martin,martin,62.0,22.0,0.354839,36,1.636364,22.0,0.611111,15.0,0.241935,13.0
max,max,0.0,0.0,0.000000,1,inf,0.0,0.000000,0.0,NaN,NaN
paddy,paddy,1.0,0.0,0.000000,1,inf,0.0,0.000000,0.0,0.000000,NaN
tony,tony,32.0,12.0,0.375000,16,1.333333,8.0,0.500000,7.0,0.218750,7.0
zed,zed,4.0,0.0,0.000000,7,inf,3.0,0.428571,0.0,0.000000,NaN


In [11]:
shuju2 = shuju.copy()
last_date = str((datetime.datetime.now()+datetime.timedelta(days=day-1)).strftime('%Y/%m/%d'))
tem = pd.DataFrame({'人员': last_date,
                    '注册数':shuju2['注册数'].sum(),
                    '绑卡数':shuju2['绑卡数'].sum(),
                    '绑卡率':shuju2['绑卡数'].sum()/shuju2['注册数'].sum(),
                    '提单人数':shuju2['提单人数'].sum(),
                    '提单率':shuju2['提单人数'].sum()/shuju2['绑卡数'].sum(),
                    '首存数':shuju2['首存数'].sum(),
                    '提单首充率':shuju2['首存数'].sum()/shuju2['提单人数'].sum(),
                    '当日注册当日首存':shuju2['当日注册当日首存'].sum(),
                    '当日注册当日首存率':shuju2['当日注册当日首存'].sum()/shuju2['注册数'].sum(),
                    '当天注册当天绑卡当天提单':shuju2['当天注册当天绑卡当天提单'].sum()},index=0)
tem

TypeError: Index(...) must be called with a collection of some kind, 0 was passed

In [ ]:
shuju2.loc[]

In [6]:
# 当日注册当日首存
merge_2_user = merge_user.loc[merge_user['注册时间']==merge_user['首存时间'],]
merge_2_grp = merge_2_user.groupby('seo变化数据团队').agg({'seo变化数据团队':len})
merge_2_grp.rename(columns={'seo变化数据团队':'当日注册当日首存'},inplace=True)
shuju = shuju.join(merge_2_grp)
shuju.fillna(0,inplace=True)
shuju['当日注册当日首存率']  = shuju['当日注册当日首存']/shuju['注册数']


print(shuju)

            人员   注册数   绑卡数       绑卡率  提单人数       提单率   首存数     提单首充率  \
人员2                                                                    
aber      aber  37.0  14.0  0.378378    25  1.785714  13.0  0.520000   
ben        ben   5.0   2.0  0.400000     4  2.000000   3.0  0.750000   
dk          dk  13.0   2.0  0.153846     6  3.000000   3.0  0.500000   
hugo      hugo  55.0  15.0  0.272727    23  1.533333  17.0  0.739130   
martin  martin  62.0  22.0  0.354839    36  1.636364  22.0  0.611111   
max        max   0.0   0.0  0.000000     1       inf   0.0  0.000000   
paddy    paddy   1.0   0.0  0.000000     1       inf   0.0  0.000000   
tony      tony  32.0  12.0  0.375000    16  1.333333   8.0  0.500000   
zed        zed   4.0   0.0  0.000000     7       inf   3.0  0.428571   

        当日注册当日首存  当日注册当日首存率  
人员2                          
aber         7.0   0.189189  
ben          2.0   0.400000  
dk           1.0   0.076923  
hugo        10.0   0.181818  
martin      15.0   0.241935

In [7]:
# 当天注册当天绑卡当天提单
merge_3_user = merge_user.loc[(merge_user['注册时间']==merge_user['首存时间'])&(~merge_user['seo变化数据团队'].isna())&(merge_user['姓名']!='-'),]
merge_3_grp = merge_3_user.groupby('seo变化数据团队').agg({'会员账号':len})
merge_3_grp.rename(columns={'会员账号':'当天注册当天绑卡当天提单'},inplace=True)
shuju = shuju.join(merge_3_grp)
shuju

,人员,注册数,绑卡数,绑卡率,提单人数,提单率,首存数,提单首充率,当日注册当日首存,当日注册当日首存率,会员账号
人员2,,,,,,,,,,,
aber,aber,37.0,14.0,0.378378,25,1.785714,13.0,0.520000,7.0,0.189189,6.0
ben,ben,5.0,2.0,0.400000,4,2.000000,3.0,0.750000,2.0,0.400000,2.0
dk,dk,13.0,2.0,0.153846,6,3.000000,3.0,0.500000,1.0,0.076923,1.0
hugo,hugo,55.0,15.0,0.272727,23,1.533333,17.0,0.739130,10.0,0.181818,9.0
martin,martin,62.0,22.0,0.354839,36,1.636364,22.0,0.611111,15.0,0.241935,13.0
max,max,0.0,0.0,0.000000,1,inf,0.0,0.000000,0.0,NaN,NaN
paddy,paddy,1.0,0.0,0.000000,1,inf,0.0,0.000000,0.0,0.000000,NaN
tony,tony,32.0,12.0,0.375000,16,1.333333,8.0,0.500000,7.0,0.218750,7.0
zed,zed,4.0,0.0,0.000000,7,inf,3.0,0.428571,0.0,0.000000,NaN


In [12]:
trade.shape

(39178, 18)

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"